In [20]:
import data_transformation
import pandas as pd
import utils

In [2]:
%load_ext autoreload
%autoreload 2

In [10]:
obs = utils.load_pickle('observations')
events_trimmed = utils.load_pickle('events_trimmed')

In [17]:
events_trimmed.available.astype(float)

0         0.0
1         0.0
2         1.0
3         1.0
4         0.0
5         1.0
6         0.0
7         1.0
8         1.0
9         1.0
10        0.0
11        1.0
12        0.0
13        1.0
14        0.0
15        0.0
16        1.0
17        1.0
18        1.0
19        0.0
20        NaN
21        1.0
22        0.0
23        NaN
24        NaN
25        0.0
26        1.0
27        0.0
28        1.0
29        NaN
         ... 
297887    1.0
297888    1.0
297889    0.0
297890    NaN
297891    1.0
297892    0.0
297893    0.0
297894    0.0
297895    1.0
297896    0.0
297897    0.0
297898    0.0
297899    NaN
297900    1.0
297901    NaN
297902    1.0
297903    0.0
297904    1.0
297905    1.0
297906    0.0
297907    0.0
297908    1.0
297909    0.0
297910    0.0
297911    0.0
297912    0.0
297913    0.0
297914    0.0
297915    1.0
297916    0.0
Name: available, Length: 297917, dtype: float64

In [3]:
# data_transformation.SESSION_TIME_LIMIT = 30
events_trimmed, item_p = data_transformation.load()

Total number of events: 2,756,101

Total number of trimmed events: 297,917



In [4]:
def add_item_property(events_df, property_df, p_name):
    
    events_df.sort_values('local_date_time', inplace=True)
    property_df.sort_values('local_date_time', inplace=True)

    events_df = pd.merge_asof(events_df, property_df[property_df.property == p_name], 
                                   on='local_date_time',
                                   by='itemid')

    events_df = events_df.rename(columns={
                                                        'timestamp_x': 'timestamp',
                                                        'value': p_name
                                                    })

    events_df = events_df.drop(columns=['timestamp_y', 'property'])
    
    return events_df

In [5]:
print(events_trimmed.shape)
events_trimmed = add_item_property(events_trimmed, item_p, 'categoryid')
events_trimmed = add_item_property(events_trimmed, item_p, 'available')
print(events_trimmed.shape)
events_trimmed.head()

(297917, 9)
(297917, 11)


,timestamp,visitorid,event,itemid,transactionid,local_date_time,minutes_since_prev_event,session_id,seq,categoryid,available
0,1441090806954,230847,view,104544,NaN,2015-09-01 00:00:06.954,0.0,230847_1,1.0,1642,0
1,1441090813080,1316340,view,126126,NaN,2015-09-01 00:00:13.080,0.0,1316340_1,1.0,646,0
2,1441090816594,1282520,view,226245,NaN,2015-09-01 00:00:16.594,0.0,1282520_1,1.0,479,1
3,1441090822265,1309256,view,365703,NaN,2015-09-01 00:00:22.265,0.0,1309256_1,1.0,589,1
4,1441090832644,217790,view,54147,NaN,2015-09-01 00:00:32.644,0.0,217790_1,1.0,65,0


In [ ]:
cat_event[cat_event['value'].isnull()]

In [ ]:
events_trimmed.shape

In [ ]:
item_counts = item_p[(item_p.property == 'categoryid')].itemid.value_counts().reset_index()
item_f = item_counts[item_counts.itemid > 1].loc[:,'index'].unique()
cat_event[cat_event.itemid.isin(item_f)].head()

In [ ]:
item_p[(item_p.itemid == 20299) & (item_p.property == 'categoryid')]

In [ ]:
cat_event[cat_event.itemid == 20299]

In [ ]:
df = events_trimmed[events_trimmed.visitorid == 152963]
df[df.seq == 1].minutes_since_prev_event.describe()

In [ ]:
observations, prior_observations = data_transformation.create_observations(events_trimmed, 2)

In [ ]:
observations.head()

In [ ]:
prior_observations.head()

In [1]:
import model_selection

In [3]:
results = model_selection.run_models()

logistic: best score: 0.7620253991219698
gb: best score: 0.851375600601442
forest: best score: 0.8117590118672657


In [10]:
import pickle

In [11]:
pickle.dump( results['gb'].best_estimator_, open( "../data/best_gb_model.pkl", "wb" ) )

In [12]:
best_model = pickle.load( open( "../data/best_gb_model.pkl", "rb" ) )

In [22]:
import utils
obs = utils.load_pickle('features')
X, y, X_train, X_test, y_train, t_test = model_selection.create_Xy(obs)

In [15]:
best_model.score(X_train, y_train)

0.7595367847411444

In [26]:
from sklearn.metrics import roc_auc_score

In [27]:
roc_auc_score(y_train, best_model.predict(X_train))

0.7595367847411444

In [28]:
results['gb'].best_score_

0.851375600601442

In [29]:
results['gb'].best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.38423011012239755, loss='exponential',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=281,
              presort='auto', random_state=1234, subsample=1.0, verbose=0,
              warm_start=False)

In [31]:
best_model.fit(X_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.38423011012239755, loss='exponential',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=281,
              presort='auto', random_state=1234, subsample=1.0, verbose=0,
              warm_start=False)

In [32]:
best_model.score(X_train, y_train)

0.7595367847411444

In [33]:
X_train.shape

(23488, 6)